# Project 1 Part 4 - Creating a master parcel database

In this part of the project, we will use Python to read, process, and double all of the parcel data into a database.  Note that this is not our only alternative, and in Project 1 Part 4 b, we will look at another alternative, that is reading all the of original, raw files into their own database table, then using SQL to join/link/aggregate the tables.

## Chunking Files in Pandas – Part 1 (20 Points)

In this part of the project, you will use `Panda`’s to process the data from the MinneMUDAC 2016 competition Dive into Water Data.  The data can be found at the [MinneMUDAC site](http://minneanalytics.org/minnemudac/data/).  You should document your work in a Jupyter notebook, which will be used to submit your solution.  **For the rest of the parts of this project, we will limit ourselves to the years 2004-2014.**

1. Remind me why we want to skip 2003.

There was not a lot of matching things in it so it made the union smaller

2. Import the common columns list and translation dictionaries from the `.py` file you created in the last part of the project.

In [2]:
from project_data_Quam import *

3. Use glob and a list comprehension to get a list of file names for the years 2004-2014.

In [3]:
from glob import glob
bad_files = ['./MinneMUDAC_raw_files/2002_metro_tax_parcels.txt', 
             './MinneMUDAC_raw_files/2003_metro_tax_parcels.txt', 
             './MinneMUDAC_raw_files/2015_metro_tax_parcels.txt',
             './MinneMUDAC_raw_files/2016_metro_tax_parcels.txt']
 

In [4]:
files = [file for file in glob('./MinneMUDAC_raw_files/20**_metro_tax_parcels.txt') if file not in bad_files ]
files[:2]

['./MinneMUDAC_raw_files/2009_metro_tax_parcels.txt',
 './MinneMUDAC_raw_files/2007_metro_tax_parcels.txt']

4. Use the first chunk of the first file to prototype an expression that <br>
    a. Selects the common columns <br>
    b. Fixes any issues with the column names <br>
    c. Changes columns to the correct types (if necessary).  More information about the columns can be found [here](ftp://ftp.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_prcls_open/metadata/metadata.html). It is **imperative** that you keep the lat and long columns as strings. <br>
    d. Use the translation dictionaries from the last part to add three new columns to the chunk: lake code, lake name, parcel distance to the lake.
    e. Filters to only properties that are within 1600 m (~1 mile) of the closest lake.

In [31]:
dtypes_dict = {'centroid_lat': str,
               'centroid_long': str}

In [6]:
import pandas as pd
from toolz import first
example_iter = pd.read_csv("./MinneMUDAC_raw_files/2005_metro_tax_parcels.txt", sep= "|", chunksize= 500, dtype=dtypes_dict)
example_chunk = first(example_iter)
len(example_chunk.columns)


70

In [7]:
from dfply import *
from more_dfply import recode, ifelse
df_ll = (example_chunk
         >> select(common_columns)
         >> mutate(lat_long = pd.Series(zip(example_chunk.centroid_lat, example_chunk.centroid_long)))
         >> mutate(lake_name = recode(X.lat_long, lat_long_name_dict),
                   lake_code = recode(X.lat_long, lat_long_code_dict),
                   distance = recode(X.lat_long, lat_long_dist))
         >> filter_by(~X.lake_name.isna())
         >> filter_by(X.distance <= 1600)
        )
df_ll

,STREETTYPE,AGPRE_EXPD,EMV_TOTAL,TAX_ADD_L1,GARAGE,STREETNAME,TAX_NAME,OWN_ADD_L1,USE4_DESC,EMV_BLDG,...,LOT,PARC_CODE,GREEN_ACRE,TAX_CAPAC,BLDG_NUM,OPEN_SPACE,lat_long,lake_name,lake_code,distance


In [8]:
len(lat_long_name_dict)

2300453

5. Now convert your expression from the last problem to a function and test that this function works on the first few chunks of each file.

In [32]:
from functoolz import pipeable
read_parcel = lambda path: pd.read_csv(path, sep= "|", chunksize= 10000, dtype=dtypes_dict)


In [10]:
add_name_code_dist = pipeable(lambda chunk:(chunk
                                            >> select(common_columns)
                                            >> mutate(lat_long = pd.Series(zip(chunk.centroid_lat, chunk.centroid_long)))
                                            >> mutate(lake_name = recode(X.lat_long, lat_long_name_dict),
                                                      lake_code = recode(X.lat_long, lat_long_code_dict),
                                                      distance = recode(X.lat_long, lat_long_dist))
                                            >> filter_by(~X.lake_name.isna())
                                            >> filter_by(X.distance <= 1600)))


In [76]:
ex = read_parcel(files[3])

In [77]:
poo =next(ex)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [79]:
poo.columns

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_long,centroid_lat
0,0.0,8.03,NaN,NaN,N,N,NaN,NaN,ST FRANCIS,NaN,...,NaN,NaN,NaN,NaN,1980.0,2005,NaN,NaN,-93.26739,45.41332
1,0.0,0.93,NaN,NaN,N,Y,24457.0,NaN,ST FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1974.0,2005,55005.0,9547.0,-93.2701,45.41354
2,0.0,8.75,NaN,NaN,N,Y,24442.0,NaN,ST FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1969.0,2005,55005.0,9547.0,-93.27344,45.41318
3,0.0,11.17,NaN,NaN,N,N,410.0,NaN,ST FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1989.0,2005,55005.0,9404.0,-93.27684,45.41167
4,0.0,14.46,NaN,NaN,N,Y,480.0,NaN,ST FRANCIS,BETHEL,...,NaN,NaN,NaN,NaN,1995.0,2005,55070.0,9404.0,-93.27849,45.41169
5,0.0,82.14,NaN,NaN,N,NaN,NaN,NaN,ST FRANCIS,NaN,...,NaN,NaN,NaN,NaN,0.0,2005,NaN,NaN,-93.29885,45.40981
6,0.0,4.69,NaN,NaN,N,Y,532.0,NaN,ST FRANCIS,ST FRANCIS,...,NaN,NaN,NaN,NaN,2004.0,2005,NaN,5070.0,-93.27973,45.41172
7,0.0,4.59,NaN,NaN,N,Y,550.0,NaN,ST FRANCIS,ISANTI,...,NaN,NaN,NaN,NaN,1969.0,2005,55040.0,4552.0,-93.28033,45.41168
8,0.0,4.28,NaN,NaN,N,NaN,NaN,NaN,ST FRANCIS,NaN,...,NaN,NaN,NaN,NaN,0.0,2005,NaN,NaN,-93.28091,45.4117
9,0.0,38.17,NaN,NaN,N,NaN,NaN,NaN,ST FRANCIS,NaN,...,NaN,NaN,NaN,NaN,0.0,2005,NaN,NaN,-93.28364,45.41162


In [16]:
example_file = add_name_code_dist(next(ex))
example_file

,STREETTYPE,AGPRE_EXPD,EMV_TOTAL,TAX_ADD_L1,GARAGE,STREETNAME,TAX_NAME,OWN_ADD_L1,USE4_DESC,EMV_BLDG,...,LOT,PARC_CODE,GREEN_ACRE,TAX_CAPAC,BLDG_NUM,OPEN_SPACE,lat_long,lake_name,lake_code,distance
280,NaN,NaN,535800.0,2340 COUNTY ROAD 127,Y,COUNTY ROAD 127,JAMES R & KAREN K HANSON,NaN,NaN,238800.0,...,NaN,1.0,N,4742.0,2340.0,NaN,"(44.96062, -93.78034)",Oak Lake,10009300-01,1282.596493
316,NaN,NaN,61400.0,2425 COUNTY ROAD 127,N,COUNTY ROAD 127,BERNARD KASTER,NaN,NaN,2900.0,...,NaN,1.0,N,333.0,2425.0,NaN,"(44.95788, -93.77517)",Oak Lake,10009300-01,1577.350617


6. We need to make a unique primary key for each row in the combined parcel file.<br>
    a. There is a column that appears to be a unique parcel id.  Double check that this is a true primary key for each individual file. (To do this you need to verify that the number of unique values is the same as the number of rows for each of the parcel files.  **Hint:** For each file, use of the accumulator pattern with two accumualtors (one number and one data frame). <br>
    b. Explain why this column will not work as a primary key if we want to combine all years in one database. <br>
    c. Suppose we make a new column that consist of `str(year) + '-' + PID`.  Explain why this should make a proper primary key for the combined data. <br>

In [85]:
example_file.BASEMENT.unique()

array([nan], dtype=object)

In [83]:
[col for num, col in enumerate(df_ll.columns) if num >= 23]

['PREFIXTYPE',
 'centroid_long',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'OWN_ADD_L2',
 'WSHD_DIST',
 'ACRES_DEED',
 'SALE_DATE',
 'COOLING',
 'BLOCK',
 'USE1_DESC',
 'ACRES_POLY',
 'XUSE4_DESC',
 'CITY',
 'SALE_VALUE',
 'FIN_SQ_FT',
 'EMV_LAND',
 'SUFFIX_DIR',
 'ZIP4',
 'TAX_EXEMPT',
 'HOME_STYLE',
 'AGPRE_ENRD',
 'GARAGESQFT',
 'MULTI_USES',
 'OWNER_MORE',
 'USE3_DESC',
 'BASEMENT',
 'PREFIX_DIR',
 'XUSE3_DESC',
 'Year',
 'centroid_lat',
 'PIN',
 'NUM_UNITS',
 'HEATING',
 'TAX_ADD_L2',
 'COUNTY_ID',
 'PLAT_NAME',
 'TAX_ADD_L3',
 'TOTAL_TAX',
 'OWNER_NAME',
 'XUSE1_DESC',
 'LOT',
 'PARC_CODE',
 'GREEN_ACRE',
 'TAX_CAPAC',
 'BLDG_NUM',
 'OPEN_SPACE',
 'lat_long',
 'lake_name',
 'lake_code',
 'distance']

In [ ]:
'PREFIXTYPE',
 'centroid_long',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'OWN_ADD_L2',
 'WSHD_DIST',
 'ACRES_DEED',
 'SALE_DATE',
 'COOLING',
 'BLOCK',
 'USE1_DESC',
 'ACRES_POLY',
 'XUSE4_DESC',
 'CITY',
 'SALE_VALUE',
 'FIN_SQ_FT',
 'EMV_LAND',
 'SUFFIX_DIR',
 'ZIP4',
 'TAX_EXEMPT',
 'HOME_STYLE',
 'AGPRE_ENRD',
 'GARAGESQFT',
 'MULTI_USES',
 'OWNER_MORE',
 'USE3_DESC',
 'BASEMENT',
 'PREFIX_DIR',
 'XUSE3_DESC',
 'Year',
 'centroid_lat',
 'PIN',
 'NUM_UNITS',
 'HEATING',
 'TAX_ADD_L2',
 'COUNTY_ID',
 'PLAT_NAME',
 'TAX_ADD_L3',
 'TOTAL_TAX',
 'OWNER_NAME',
 'XUSE1_DESC',
 'LOT',
 'PARC_CODE',
 'GREEN_ACRE',
 'TAX_CAPAC',
 'BLDG_NUM',
 'OPEN_SPACE',
 'lat_long',
 'lake_name',
 'lake_code',
 'distance'

In [34]:
unique_pins = set([])
rows = 0
for file in files:
    print("\n\n\n Starting to process " + file)
    for i, chunk in enumerate(read_parcel(file)):
        #processed_chunk = add_name_code_dist(chunk)
        rows = rows + len(chunk.PIN)
        unique_pins = unique_pins.union(chunk.PIN)
        print("On chunk" + str(i) + " : amount of unique pins = " + str(len(unique_pins)) + " ,     total amount = " + str(rows))




 Starting to process ./MinneMUDAC_raw_files/2009_metro_tax_parcels.txt
On chunk0 : amount of unique pins = 9985 ,     total amount = 10000


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On chunk1 : amount of unique pins = 19929 ,     total amount = 20000
On chunk2 : amount of unique pins = 29884 ,     total amount = 30000


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On chunk3 : amount of unique pins = 39839 ,     total amount = 40000
On chunk4 : amount of unique pins = 49728 ,     total amount = 50000
On chunk5 : amount of unique pins = 59683 ,     total amount = 60000
On chunk6 : amount of unique pins = 69620 ,     total amount = 70000
On chunk7 : amount of unique pins = 79533 ,     total amount = 80000
On chunk8 : amount of unique pins = 89391 ,     total amount = 90000


...

On chunk169 : amount of unique pins = 1282253 ,     total amount = 22423159
On chunk170 : amount of unique pins = 1282253 ,     total amount = 22433159
On chunk171 : amount of unique pins = 1282253 ,     total amount = 22443159
On chunk172 : amount of unique pins = 1282253 ,     total amount = 22453159


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (33,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On chunk173 : amount of unique pins = 1282253 ,     total amount = 22463159
On chunk174 : amount of unique pins = 1282253 ,     total amount = 22473159
On chunk175 : amount of unique pins = 1282253 ,     total amount = 22483159
On chunk176 : amount of unique pins = 1282253 ,     total amount = 22493159
On chunk177 : amount of unique pins = 1282253 ,     total amount = 22503159
On chunk178 : amount of unique pins = 1282253 ,     total amount = 22513159


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,10,12,17,19,20,26,37,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On chunk179 : amount of unique pins = 1282253 ,     total amount = 22523159
On chunk180 : amount of unique pins = 1282253 ,     total amount = 22533159
On chunk181 : amount of unique pins = 1282253 ,     total amount = 22543159
On chunk182 : amount of unique pins = 1282253 ,     total amount = 22553159
On chunk183 : amount of unique pins = 1282253 ,     total amount = 22563159
On chunk184 : amount of unique pins = 1282253 ,     total amount = 22573159
On chunk185 : amount of unique pins = 1282253 ,     total amount = 22583159
On chunk186 : amount of unique pins = 1282253 ,     total amount = 22593159
On chunk187 : amount of unique pins = 1282253 ,     total amount = 22603159
On chunk188 : amount of unique pins = 1282253 ,     total amount = 22613159
On chunk189 : amount of unique pins = 1282253 ,     total amount = 22623159
On chunk190 : amount of unique pins = 1282253 ,     total amount = 22633159
On chunk191 : amount of unique pins = 1282253 ,     total amount = 22643159
On chunk192 

7. Make a function to add the key suggested in the last problem (`str(year) + '-' + PID`) to a given chunk.

In [51]:
example_chunk.dtypes['PIN']

dtype('O')

In [53]:
add_primary_key = pipeable(lambda start, df: (df
                                              >> mutate(id = np.arange(start, start + len(df))
                                              )))

#### Note: If you are clever, you can do parts 8 in one double loop, which will save you from having to read the parcel files twice.

8. It is probably worth our time to test that our new key column is truely unique. (If not, we might be wasting out time loading the data into a database, only to have process fail hours in.) Test that the new column works by <br>
    a. Iterating over all the files.<br>
    b. Using an accumulator to count total number of rows across all parcel files. <br>
    c. Using an accumulator to accumulate a set of all unique values of our new key. <br>
    d. Verifying that we have as many total rows as unique keys.
    a. Selecting just this column. <br>
    b. Dumping this column into a temporary database <br>

In [61]:
unique_ids = set([])
rows = 0
rows_so_far = 0
for file in files:
    print("\n\n\n Starting to process " + file)
    c = 0
    for i, chunk in enumerate(read_parcel(file)):
        while c <= 2 :
            c = c+1
            processed_chunk = add_primary_key(rows_so_far,chunk)
            rows = rows + len(processed_chunk.id)
            unique_ids = unique_ids.union(processed_chunk.id)
            print("On chunk" + str(i) + " : amount of unique ids = " + str(len(unique_ids)) + " ,     total amount = " + str(rows))
            rows_so_far = rows_so_far + len(chunk)
        




 Starting to process ./MinneMUDAC_raw_files/2009_metro_tax_parcels.txt


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On chunk0 : amount of unique ids = 10000 ,     total amount = 10000
On chunk0 : amount of unique ids = 20000 ,     total amount = 20000
On chunk0 : amount of unique ids = 30000 ,     total amount = 30000


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,19,21,27,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anacond

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,10,17,21,23,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,59,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,56,60,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, r

...

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32,49,54,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,32,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,32,49,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,32,49,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result




 Starting to process ./MinneMUDAC_raw_files/2004_metro_tax_parcels.txt
On chunk0 : amount of unique ids = 310000 ,     total amount = 310000
On chunk0 : amount of unique ids = 320000 ,     total amount = 320000
On chunk0 : amount of unique ids = 330000 ,     total amount = 330000


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (38,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (25,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/pyth

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (25,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (25,33,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (33,37,38,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (33,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (33,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaco

In [55]:
add_primary_key(0,example_chunk)

,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,...,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_long,centroid_lat,id
0,0.0,8.03,NaN,NaN,N,N,NaN,NaN,ST FRANCIS,NaN,...,NaN,NaN,NaN,1980.0,2005,NaN,NaN,-93.26739,45.41332,0
1,0.0,0.93,NaN,NaN,N,Y,24457.0,NaN,ST FRANCIS,BETHEL,...,NaN,NaN,NaN,1974.0,2005,55005.0,9547.0,-93.2701,45.41354,1
2,0.0,8.75,NaN,NaN,N,Y,24442.0,NaN,ST FRANCIS,BETHEL,...,NaN,NaN,NaN,1969.0,2005,55005.0,9547.0,-93.27344,45.41318,2
3,0.0,11.17,NaN,NaN,N,N,410.0,NaN,ST FRANCIS,BETHEL,...,NaN,NaN,NaN,1989.0,2005,55005.0,9404.0,-93.27684,45.41167,3
4,0.0,14.46,NaN,NaN,N,Y,480.0,NaN,ST FRANCIS,BETHEL,...,NaN,NaN,NaN,1995.0,2005,55070.0,9404.0,-93.27849,45.41169,4
5,0.0,82.14,NaN,NaN,N,NaN,NaN,NaN,ST FRANCIS,NaN,...,NaN,NaN,NaN,0.0,2005,NaN,NaN,-93.29885,45.40981,5
6,0.0,4.69,NaN,NaN,N,Y,532.0,NaN,ST FRANCIS,ST FRANCIS,...,NaN,NaN,NaN,2004.0,2005,NaN,5070.0,-93.27973,45.41172,6
7,0.0,4.59,NaN,NaN,N,Y,550.0,NaN,ST FRANCIS,ISANTI,...,NaN,NaN,NaN,1969.0,2005,55040.0,4552.0,-93.28033,45.41168,7
8,0.0,4.28,NaN,NaN,N,NaN,NaN,NaN,ST FRANCIS,NaN,...,NaN,NaN,NaN,0.0,2005,NaN,NaN,-93.28091,45.4117,8
9,0.0,38.17,NaN,NaN,N,NaN,NaN,NaN,ST FRANCIS,NaN,...,NaN,NaN,NaN,0.0,2005,NaN,NaN,-93.28364,45.41162,9


In [64]:
example_chunk.AGPRE_ENRD


0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
29    NaN
       ..
470   NaN
471   NaN
472   NaN
473   NaN
474   NaN
475   NaN
476   NaN
477   NaN
478   NaN
479   NaN
480   NaN
481   NaN
482   NaN
483   NaN
484   NaN
485   NaN
486   NaN
487   NaN
488   NaN
489   NaN
490   NaN
491   NaN
492   NaN
493   NaN
494   NaN
495   NaN
496   NaN
497   NaN
498   NaN
499   NaN
Name: AGPRE_ENRD, Length: 500, dtype: float64

9. If the last step succeeded, you can proceed to make a master parcel data database.  If not, you will need to figure out another primary key, probably an `id` column similar to the example in the lectures.